In [1]:
import geopandas as gpd
import requests
from PIL import Image
import numpy as np
import pandas as pd


In [ ]:

# Wczytaj granice świata z pliku zip
world = gpd.read_file("../ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

poland = world[world['NAME'] == 'Poland']
poland_merc = poland.to_crs(epsg=3857)
xmin, ymin, xmax, ymax = poland_merc.total_bounds
print(f"{xmin=}, {ymin=}, {xmax=}, {ymax=}")


In [ ]:
xmin = 2322169
ymin = 6744338
xmax = 2407567
ymax = 6844578
bbox = f"{xmin},{ymin},{xmax},{ymax}"


params = {
    "bbox": bbox,
    "bboxSR": "102100",     # EPSG:3857 (Web Mercator)
    "imageSR": "102100",
    "size": "2048,2048",    # rozdzielczość a
    "format": "png",        # 'jpg', 'tiff'
    "f": "image"
}
url = "https://image.discomap.eea.europa.eu/arcgis/rest/services/Corine/CLC1990_WM/MapServer/export"
# 1990,2000,2006,2012,2018

response = requests.get(url, params=params)

with open("clc_warszawa_2012.png", "wb") as f:
    f.write(response.content)

print("Obraz zapisany jako")


In [ ]:

img = Image.open("clc2018_polska.png").convert("RGB")
img_array = np.array(img)

height, width, _ = img_array.shape

pixels = img_array.reshape(-1, 3)
rows = np.repeat(np.arange(height), width)
cols = np.tile(np.arange(width), height)

df = pd.DataFrame({
    'x': cols,
    'y': rows,
    'R': pixels[:, 0],
    'G': pixels[:, 1],
    'B': pixels[:, 2]
})

print(df)


In [ ]:
# Mapa kolorów do klas 
color_to_clc = {
    (230, 0, 77): '111 - Zabudowa ciągła',
    (255, 0, 0): '112 - Zabudowa rozproszona',
    (166, 0, 255): '121 - Obszary przemysłowe',
    (255, 255, 100): '211 - Grunty orne',
    (0, 255, 0): '313 - Lasy mieszane',
    (0, 0, 255): '511 - Wody śródlądowe',
    (0, 128, 0): '312 - Lasy liściaste',
    (255, 255, 0): '222 - Uprawy trwałe',
    (255, 245, 215): '331 - Piaski',
}



In [ ]:

df["rgb"] = list(zip(df["R"], df["G"], df["B"]))
df['class'] = df['rgb'].map(color_to_clc).fillna('Nieznana')
print(df["class"].value_counts())

class_counts = df["class"].value_counts(normalize=True) * 100
print(class_counts)
